In [1]:
import pandas as pd
import geopandas as gpd
import folium
from shapely import wkt
from shapely.geometry import LineString

## Read Simplified Freeway Network Dataset

In [100]:
def create_network_geodataframe(file_path,crs=2230):
    """
    Creates network geodataframe from csv with geometry in well-known text format. 
    Required columns = ['A','B','FT','GEOMETRY']
    
    Parameters
    -------------------
    file_path (String):
    Path to csv
    
    crs (String):
    Coordinate reference system as a string, such as an authority string (eg "EPSG:4326")
    
    Returns
    -------------------
    Geodataframe object
    """
    req_cols = ['A','B','FT','GEOMETRY']
    #Read csv 
    nw_df = pd.read_csv('simple_network.csv')
    
    #Capitalize all column names
    nw_df.columns = [x.upper() for x in nw_df.columns]
    
    #Check if required columns existing in csv
    if all([item in nw_df.columns for item in req_cols]):
        #Keep only required columns
        nw_df = nw_df[req_cols]
        #Create LINK_ID column. Links are uniquely identified by A and B nodes.
        nw_df['LINK_ID'] = nw_df['A'].astype(str) + '-' + nw_df['B'].astype(str)
        #Create coordinates geoseries from wkt
        coordinates = gpd.GeoSeries.from_wkt(nw_df['GEOMETRY'],crs='EPSG:2230')
        #Create geodataframe using geoseries coordinates as geometry
        nw_gdf = gpd.GeoDataFrame(nw_df,geometry=coordinates)
        #Drop wkt representation of geometry
        nw_gdf = nw_gdf.drop(columns='GEOMETRY')
        return nw_gdf
    else:
       print(f"Required columns not present. \nRequired columns: {req_cols}")

In [101]:
create_network_geodataframe(file_path='simple_network.csv')

,A,B,FT,LINK_ID,geometry
0,2000020,2129611,1.0,2000020-2129611,"LINESTRING (4869374.374 3570918.140, 4869362.7..."
1,3000068,3013888,1.0,3000068-3013888,"LINESTRING (4891305.524 3814741.652, 4891452.8..."
2,2000282,2132812,2.0,2000282-2132812,"LINESTRING (4895582.051 3570337.754, 4895630.9..."
3,3000175,3018779,1.0,3000175-3018779,"LINESTRING (4811384.272 3813607.104, 4811221.0..."
4,2000551,2121735,1.0,2000551-2121735,"LINESTRING (4927513.611 3537477.980, 4928075.1..."
...,...,...,...,...,...
744613,2154279,2192760,5.0,2154279-2192760,"LINESTRING (5044369.823 3429431.733, 5043912.8..."
744614,2192760,2154279,5.0,2192760-2154279,"LINESTRING (5043912.889 3430224.962, 5044369.8..."
744615,2137905,2129456,5.0,2137905-2129456,"LINESTRING (5042027.266 3433287.919, 5037724.3..."
744616,2129456,2137905,5.0,2129456-2137905,"LINESTRING (5037724.342 3433803.729, 5042027.2..."


In [6]:
#Filter nw gdf to a gdf containing only freeways
fw_nw_gdf = nw_gdf[nw_gdf['FT'] == 1].copy()

In [7]:
#Filter new gdf to a gdf containing only ramps
rmp_nw_gdf = nw_gdf[nw_gdf['FT'] == 3].copy()

## Find intersections using geopandas overlay

In [8]:
#Find intersections, keeping only point geom types
intersection = gpd.overlay(fw_nw_gdf,rmp_nw_gdf,
                            how='intersection',
                            keep_geom_type=False)

In [9]:
#Filter for point geometries
intersection_pts = intersection[intersection.geom_type == 'Point'].copy()

In [11]:
#export for inspection
intersection_pts.to_file('intersection_pts.geojson',driver='GeoJSON')

In [12]:
interchanges = intersection_pts[(intersection_pts['A_1'] == intersection_pts['B_2']) | 
                          (intersection_pts['B_1'] == intersection_pts['A_2'])].copy()

In [13]:
#export interchanges for inspection
interchanges.to_file('interchanges.geojson',driver='GeoJSON')

In [14]:
interchanges

,A_1,B_1,FT_1,link_id_1,A_2,B_2,FT_2,link_id_2,geometry
0,2000020,2129611,1,2000020-2129611,2090716,2000020,3,2090716-2000020,POINT (4869374.374 3570918.140)
2,2000020,2129611,1,2000020-2129611,2129611,2054853,3,2129611-2054853,POINT (4869549.739 3574718.303)
5,3013888,3023480,1,3013888-3023480,3020740,3013888,3,3020740-3013888,POINT (4896265.634 3815485.212)
6,3000175,3018779,1,3000175-3018779,3018779,3053273,3,3018779-3053273,POINT (4809637.428 3812571.989)
8,3000175,3018779,1,3000175-3018779,3061783,3000175,3,3061783-3000175,POINT (4811384.272 3813607.104)
...,...,...,...,...,...,...,...,...,...
5403,1547936,1547976,1,1547936-1547976,1547976,1505692,3,1547976-1505692,POINT (4776244.348 3680925.704)
5405,2158847,2067909,1,2158847-2067909,2067909,2067006,3,2067909-2067006,POINT (4896489.452 3536411.118)
5409,1547976,1544865,1,1547976-1544865,1544865,1534615,3,1544865-1534615,POINT (4776030.165 3681910.965)
5410,1548379,1501662,1,1548379-1501662,1547614,1548379,3,1547614-1548379,POINT (4758377.660 3696272.460)


In [15]:
#Flag on-ramp and off-ramp
#Freeway network was intersected with ramp network. 
#Ramps are classified as on ramp if A_1 = B_2, and off if B_1 = A_2. 
interchanges['ramp_type'] = interchanges.apply(lambda row: "on" if row['A_1'] == row['B_2'] else "off",axis=1)

In [43]:
def is_node_connected_to_freeway(row,nw_gdf):
    #Determine direction by ramp (on/off)
    #if onramp, we would look backwards starting with A. if off, look foward starting with B.
    #if empty set returned, set false
    #if any result is a freeway set true
    #if all results are not freeway or ramp set false
    #Recurse
    ##If any result is a ramp, filter for ramp - call validator 
    
    nw_ramp_gdf = nw_gdf[nw_gdf['link_id'] == row['link_id_2']].iloc[0]
    ramp_type = row['ramp_type']
    #freeway_name = row['fwy_name']
    inspected_ramps = []
    
    def validator(nw_ramp_gdf,ramp_type,inspected_ramps):
        link_id = nw_ramp_gdf['link_id']
        ramp_start = nw_ramp_gdf['A']
        ramp_end = nw_ramp_gdf['B']
        inspected_ramps.append(link_id)
        
        if ramp_type == 'on':
            #Check if ramp is connected to any other links
            if ramp_start not in nw_gdf.B.values:
                return False
            #If connected to another link, check if link is FT = 1 (Freeway) 
            #To improve this, we need more attributes about the network, such as name of fwy
            #then we could check that the ramp is not just connected to a fwy, but a fwy 
            #of another name. 
            elif 1 in nw_gdf.loc[nw_gdf['B'] == ramp_start,'FT'].values:
                return True
            #If not connected to another ramp link, then not an interchange
            elif 3 not in nw_gdf.loc[nw_gdf['B'] == ramp_start,'FT'].values:
                return False
            #If it is connected to another ramp link, check all ramp links using recursion 
            #to determine if they are connected to a freeway and stop 
            #looking to see if we've already looked at the ramp link.
            else:
                #Look at network for ramp ends, connecting to ramp start from interchanges
                #Make sure that network ramp starts, do not cycle with ramp end 
                #(Case where freeway exit is also onramp) A != B, B != A
                connecting_ramps = nw_gdf[(nw_gdf['B'] == ramp_start) & 
                                          (~nw_gdf['link_id'].isin(inspected_ramps)) &
                                          (nw_gdf['FT'] == 3)]
                return any([validator(row, ramp_type,inspected_ramps) for index,row in connecting_ramps.iterrows()])
        else:
            if ramp_end not in nw_gdf.A.values:
                return False
            elif 1 in nw_gdf.loc[nw_gdf['A'] == ramp_end,'FT'].values:
                return True
            elif 3 not in nw_gdf.loc[nw_gdf['A'] == ramp_end,'FT'].values:
                return False
            else:
                connecting_ramps = nw_gdf[(nw_gdf['A'] == ramp_end) & 
                                          (~nw_gdf['link_id'].isin(inspected_ramps)) &
                                          (nw_gdf['FT'] == 3)]
                return any([validator(row, ramp_type,inspected_ramps) for index,row in connecting_ramps.iterrows()])
    return validator(nw_ramp_gdf,ramp_type,inspected_ramps)

In [44]:
interchanges['is_fwy_interchange'] = (interchanges
                                      .apply(lambda row: is_node_connected_to_freeway(row,nw_gdf),axis=1))    

In [46]:
interchanges[interchanges['is_fwy_interchange']].explore()

In [49]:
interchanges[interchanges['is_fwy_interchange'] == True].to_file('interchanges_final.geojson',driver='GeoJSON')

In [48]:
interchanges

,A_1,B_1,FT_1,link_id_1,A_2,B_2,FT_2,link_id_2,geometry,ramp_type,is_fwy_interchange
0,2000020,2129611,1,2000020-2129611,2090716,2000020,3,2090716-2000020,POINT (4869374.374 3570918.140),on,False
2,2000020,2129611,1,2000020-2129611,2129611,2054853,3,2129611-2054853,POINT (4869549.739 3574718.303),off,True
5,3013888,3023480,1,3013888-3023480,3020740,3013888,3,3020740-3013888,POINT (4896265.634 3815485.212),on,True
6,3000175,3018779,1,3000175-3018779,3018779,3053273,3,3018779-3053273,POINT (4809637.428 3812571.989),off,True
8,3000175,3018779,1,3000175-3018779,3061783,3000175,3,3061783-3000175,POINT (4811384.272 3813607.104),on,True
...,...,...,...,...,...,...,...,...,...,...,...
5403,1547936,1547976,1,1547936-1547976,1547976,1505692,3,1547976-1505692,POINT (4776244.348 3680925.704),off,True
5405,2158847,2067909,1,2158847-2067909,2067909,2067006,3,2067909-2067006,POINT (4896489.452 3536411.118),off,True
5409,1547976,1544865,1,1547976-1544865,1544865,1534615,3,1544865-1534615,POINT (4776030.165 3681910.965),off,True
5410,1548379,1501662,1,1548379-1501662,1547614,1548379,3,1547614-1548379,POINT (4758377.660 3696272.460),on,False
